In [ ]:
from spore.mock.convert_sim_to_vis import get_cut_box
from scipy.interpolate import RectBivariateSpline
from spore.measure import unit_conversions as uc

def get_power_lightcone_2d(fname="/home/steven/Documents/Projects/Foregrounds/delta_T_v3_no_halos__zstart005.00000_zend009.56801_FLIPBOXES0_1024_1600Mpc_lighttravel",
                        numin=150.,numax=161.15):
    box,N,L,d,nu,z = get_cut_box(fname,numin,numax)
    
    pk, kperp, kpar = get_power(box,L,bins=50,res_ndim=2)
    
    return pk, kperp, kpar


def get_1d_power_at_single_z_2d(fname="/home/steven/Documents/Projects/Foregrounds/delta_T_v3_no_halos_z008.09_nf0.517893_useTs1_NX2.0e-01_alphaX1.5_TvirminX2.0e+04_aveTb009.93_Pop2_1024_1600Mpc"):
    box = readbox(fname)
    N = box.dim
    L = box.param_dict['BoxSize']
    box = box.box_data

    # Restrict the box for memory
#     box = box[:N/2,:N/2,:N/2]
#     L /=2
    
    pk = [0]*len(box)
    for i,bx in enumerate(box):
        pk[i],kbins = get_power(bx,L,bins=50)
        
    return pk, kbins


def get_power_lightcone_beam(fname, numin=150., numax=161.15, get_delta=True, bins=50, beam_model = CircularGaussian):
    box, N, L, d, nu, z = get_cut_box(fname, numin, numax)

    # INITIALISE A BEAM MODEL
    beam = beam_model(nu.min(), np.linspace(1, nu.max()/nu.min(), len(nu)))

    # ATTENUATE BOX BY THE BEAM
    width = L/Planck15.angular_diameter_distance(z).value
    attenuation = np.ones_like(box)

    for i in tqdm.tqdm(range(len(nu))):
        dl = width[i]/N

        l = np.sin(np.linspace(-width[i]/2 + dl/2, width[i]/2 - dl/2, N))
        X, M = np.meshgrid(l, l)

        attenuation[:, :, i] = np.exp(-(X ** 2 + M ** 2)/(2*beam.sigma[i] ** 2))
        box[:, :, i] = box[:, :, i]*attenuation[:, :, i]

    volfrac = np.sum(attenuation)**2 /(np.sum(attenuation**2) * np.product(attenuation.shape))

    pk, kbins = get_power(box, [L, L, np.abs(d[-1] - d[0])], bins=bins)
    #bk, kbins = get_power(attenuation, [L, L, np.abs(d[-1] - d[0])], bins=bins)
    
    if get_delta:
        pk *= kbins**3/(2*np.pi**2)
        
    return pk, kbins, bk


def get_power_lightcone_beam_uspace(fname, numin=150., numax=161.15, get_delta=True, bins=50, beam_model = CircularGaussian):
    box, N, L, d, nu, z = get_cut_box(fname, numin, numax)

    # Just for speed in testing.
    box = box[:N/2,:N/2]
    N /= 2
    L /= 2
    
    
    # INITIALISE A BEAM MODEL
    beam_model = beam_model(nu.min(), np.linspace(1, nu.max()/nu.min(), len(nu)))

    # GENERATE BASELINE VECTORS
    uv = np.linspace(-100.,100.,N)   #change N back to 400 or so
    uvsample = np.zeros((len(uv),len(uv),len(nu)),dtype='complex128')
    
    # GENERATE l,m COORDS OF BOX
    width = L/Planck15.comoving_distance(z).value

    beam = np.ones_like(box)
    for i in tqdm.tqdm(range(len(nu))):
        dtheta = width[i]/N

        # Actual coords of this slice
        l = np.sin(np.arange(-width[i]/2, width[i]/2, dtheta)[:N])
        X, M = np.meshgrid(l, l)

        # Beam for this slice
        beam[:,:,i] = np.exp(-(X ** 2 + M ** 2)/(2*beam_model.sigma[i] ** 2))

        slice = box[:,:,i]*beam[:,:,i]
        
        # Interpolate onto regular l,m grid
        spl_lm = RectBivariateSpline(l,l,slice)
        
        dl = (l.max() - l.min())/N

        l = np.arange(l.min(),l.max(),dl)[:N]

        slice = spl_lm(l,l,grid=True)

        FT, freq = fft(slice, L=l.max()-l.min(), a=0, b=2*np.pi)                  #### CHANGED L,a,b here!

        spl_rl = RectBivariateSpline(freq[0], freq[1], np.real(FT))
        spl_im = RectBivariateSpline(freq[0], freq[1], np.imag(FT))

        uvsample[:,:, i] = spl_rl(uv, uv, grid=True) + 1j*spl_im(uv, uv, grid=True)
#        uvsample[:,:,i] = FT
    
    volfrac = np.sum(beam)/np.product(beam.shape)
    
    uv = freq[0]    #### CHANGED THIS (JUST DELETE LATER)
    
    # Do the FT in the nu plane.
    ps = np.zeros_like(uvsample)
    for i in tqdm.tqdm(range(len(uv))):
        for j in range(len(uv)):
            ps[i,j], eta = fft(uvsample[i,j], L=nu.max()-nu.min(), a=0., b=2*np.pi)

    # Generate PS from FT
    ps = np.real(np.abs(ps)**2)
    ps /= L*L*(d.max()-d.min())
    
    # Perform spherical averaging.
    z = (1420.0/nu.min()) - 1
    kperp = (uv/uc.un.rad).to(uc.hub / uc.un.Mpc, equivalencies=uc.cosmo_21cm_angle_equiv(z)).value
    kpar = (eta[0]/uc.un.Hz/1e6).to(uc.hub / uc.un.Mpc, equivalencies=uc.cosmo_21cm_los_equiv(z)).value
    
    ps *= (uc.un.MHz.to(uc.un.Mpc, equivalencies=uc.cosmo_21cm_los_equiv(z)))**2              # LOS conversion
    ps *= (uc.un.steradian.to(uc.un.Mpc**2, equivalencies=uc.cosmo_21cm_angle_equiv(z)))**2   # Angle conversion
    
    #kperp = freq[0]
    #kpar = eta[0]
    KPERPx,KPERPy,KPAR = np.meshgrid(kperp,kperp,kpar)
    coords = np.sqrt(KPERPx**2 + KPERPy**2 + KPAR**2)
    
    pk, kbins = angular_average(ps, coords, 100)

    #ps = ps * uc.un.Jy**2 * uc.un.MHz**2
    #ps = uc.jyhz_to_mKMpc(ps, nu*uc.un.MHz, Aeff=20., verbose=False)

    if get_delta:
        pk *= kbins**3/(2*np.pi**2)
        
    return pk/volfrac, kbins, {"d":d,"z":z,"nu":nu,"l":l}

In [ ]:
#pk_z8, kbins_z8 = pssim.get_power_single_z("/home/steven/Documents/Projects/Foregrounds/delta_T_v3_no_halos_z008.09_nf0.517893_useTs1_NX2.0e-01_alphaX1.5_TvirminX2.0e+04_aveTb009.93_Pop2_1024_1600Mpc", maxN=512)
#pk_lc, kbins_lc = pssim.get_power_lightcone("/home/steven/Documents/Projects/Foregrounds/delta_T_v3_no_halos__zstart005.00000_zend009.56801_FLIPBOXES0_1024_1600Mpc_lighttravel")
#pk_lcb, kbins_lcb, volfrac = pssim.get_power_lightcone_beam("/home/steven/Documents/Projects/Foregrounds/delta_T_v3_no_halos__zstart005.00000_zend009.56801_FLIPBOXES0_1024_1600Mpc_lighttravel",bins=100)
pk_lcu, kbins_lcu, stuff = get_power_lightcone_beam_uspace("/home/steven/Documents/Projects/Foregrounds/delta_T_v3_no_halos__zstart005.00000_zend009.56801_FLIPBOXES0_1024_1600Mpc_lighttravel",bins=100)

andre_power = np.genfromtxt("/home/steven/Documents/Projects/Foregrounds/ps_no_halos_z008.09_nf0.517893_useTs1_NX2.0e-01_alphaX1.5_TvirminX2.0e+04_aveTb009.93_Pop2_1024_1600Mpc_v3")

In [ ]:
plt.plot(kbins_z8,pk_z8 )
plt.plot(kbins_lc,pk_lc )
plt.plot(kbins_lcb,pk_lcb/volfrac)
plt.plot(kbins_lcu,pk_lcu )

plt.plot(andre_power[:,0],andre_power[:,1])

plt.xscale('log')
plt.yscale('log')

#### Create/Test Synthetic Observation

In [ ]:
FORCE_REDO = True

if not FORCE_REDO:
    with open("sim_to_vis_MWA.pkl") as f:
        dct = pickle.load(f)
        uvsample_mwa = dct['uvsample']
        baselines_mwa = dct['baselines']
        nu1 = dct['nu']

    with open("ps_from_vis_MWA.pkl") as f:
        dct = pickle.load(f)
        ps_2d_mwa = dct['ps_2d']
        kperp = dct['kperp']
        kpar = dct['kpar']
        ubins = dct['ubins']
else:
    bl_fname = "/home/steven/Documents/Projects/Foregrounds/clustering_counts_paper/hex_pos.txt"
    box_fname = "/home/steven/Documents/Projects/Foregrounds/delta_T_v3_no_halos__zstart005.00000_zend009.56801_FLIPBOXES0_1024_1600Mpc_lighttravel"
    
    hexpos = np.genfromtxt(bl_fname)
    uvsample_mwa, baselines_mwa, nu1,tmp_data = sim_to_vis(box_fname,hexpos[:,1:], numin=150., numax=160.)
    
    bl_mag_mwa = np.sqrt(baselines_mwa[:,0]**2 + baselines_mwa[:,1]**2)
    
    with open("sim_to_vis_MWA.pkl",'w') as f:
        pickle.dump({"uvsample":uvsample_mwa,"baselines":baselines_mwa,"nu":nu1},f)

    ps_2d_mwa, kperp, kpar, ubins = power_spec_from_visibility(uvsample_mwa,baselines_mwa,nu1,n_u=1200, taper = blackmanharris, umin=2 * bl_mag_mwa.min())
    
    with open("ps_from_vis_MWA.pkl",'w') as f:
        pickle.dump({"ps_2d":ps_2d_mwa,"kperp":kperp,"kpar":kpar,"ubins":ubins},f)

##### Plot 2D power

In [ ]:
plot_2D_PS(ps_2d_mwa,kperp,kpar)
plt.ylim(kpar.min().value,kpar.max().value)
plt.show()

In [ ]:
# Get 1D power spectrum
KPERP,KPAR = np.meshgrid(kperp.value,kpar.value)
coords = np.sqrt(KPERP**2+KPAR**2)
p1d,k = angular_average(ps_2d_mwa.value,coords,30)

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(k*Planck15.h, p1d*k**3/(2*np.pi**2),label="Simulation Through Pipeline")
plt.plot(kbins_z8,pk_z8 * kbins_z8**3/(2*np.pi**2),label=r"Simulation at $\nu=155$MHz")
plt.plot(kbins_lc,pk_lc * kbins_lc**3/(2*np.pi**2),label=r"Lightcone between $\nu\in (150,160)$MHz")
plt.plot(andre_power[:,0],andre_power[:,1],label=r"Simulation at $\nu=155$ MHz (Andres PS)")
plt.plot(kbins_lcb,pk_lcb * kbins_lcb**3/(2*np.pi**2)/volfrac, label="Lightcone with Gaussian Beam")

#plt.fill_between(andre_zhi[:,0],andre_zlo[:,1],andre_zhi[:,1],color='k',alpha=0.3)
#plt.plot(andre_zlo[:,0],andre_zlo[:,1])

plt.xscale('log')
plt.yscale('log')
plt.ylim(1e-1,)
plt.xlabel(r"k, [Mpc$^{-1}$]")
plt.ylabel(r"\Delta^2(k), [mK$^2$]")
plt.legend()